In [1]:
from torchvision.models import vgg16,VGG16_Weights
import torch.nn as nn
from torchinfo import summary

model=vgg16(weights=VGG16_Weights.IMAGENET1K_V1 )


In [2]:
summary(model,input_size=(1,3,224,224))

Layer (type:depth-idx)                   Output Shape              Param #
VGG                                      [1, 1000]                 --
├─Sequential: 1-1                        [1, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [1, 64, 224, 224]         1,792
│    └─ReLU: 2-2                         [1, 64, 224, 224]         --
│    └─Conv2d: 2-3                       [1, 64, 224, 224]         36,928
│    └─ReLU: 2-4                         [1, 64, 224, 224]         --
│    └─MaxPool2d: 2-5                    [1, 64, 112, 112]         --
│    └─Conv2d: 2-6                       [1, 128, 112, 112]        73,856
│    └─ReLU: 2-7                         [1, 128, 112, 112]        --
│    └─Conv2d: 2-8                       [1, 128, 112, 112]        147,584
│    └─ReLU: 2-9                         [1, 128, 112, 112]        --
│    └─MaxPool2d: 2-10                   [1, 128, 56, 56]          --
│    └─Conv2d: 2-11                      [1, 256, 56, 56]          29

In [7]:
model.classifier[6]=nn.Linear(4096,2)
for x,y in model.named_parameters():
    for epoch in range(5):
        if 'feature' or (f'classifier' in x and not 'classifier.6.' in x):
            y.requires_grad=False
        if  'classifier.6.' in x:
            y.requires_grad=True
    
        


In [8]:
for x,y in model.named_parameters():
    print(x,y.requires_grad)

features.0.weight False
features.0.bias False
features.2.weight False
features.2.bias False
features.5.weight False
features.5.bias False
features.7.weight False
features.7.bias False
features.10.weight False
features.10.bias False
features.12.weight False
features.12.bias False
features.14.weight False
features.14.bias False
features.17.weight False
features.17.bias False
features.19.weight False
features.19.bias False
features.21.weight False
features.21.bias False
features.24.weight False
features.24.bias False
features.26.weight False
features.26.bias False
features.28.weight False
features.28.bias False
classifier.0.weight False
classifier.0.bias False
classifier.3.weight False
classifier.3.bias False
classifier.6.weight True
classifier.6.bias True


In [24]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import PIL as pil
import torch.nn as nn
from torch.utils.data import DataLoader
from torchmetrics.classification import Accuracy 
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau 
import torch
ROOT='../DATA/image/pet/'
BATCH=5
EPOCH=5
DEVICE=''if torch.cuda.is_available() else 'cpu'
transform=transforms.Compose([transforms.Resize((256,256),interpolation=transforms.InterpolationMode.BILINEAR),transforms.ToTensor(),transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))])
train_dataset=ImageFolder(ROOT+'train/',transform)
test_dataset=ImageFolder(ROOT+'test/',transform)
train_dataloader=DataLoader(train_dataset,BATCH,shuffle=True,drop_last=True)
test_dataloader=DataLoader(test_dataset,BATCH,shuffle=True,drop_last=True)

In [25]:


acurracy=Accuracy('multiclass',num_classes=2)
optimizer=optim.Adam(model.parameters(),lr=0.001)
LR=ReduceLROnPlateau(optimizer,patience=10)
model.to(DEVICE)
for epo in range(EPOCH):
    model.train()
    fact_list=[]
    pred_list=[]
    loss_list=[]
    for x,y in train_dataloader:
        x,y=x.to(DEVICE),y.to(DEVICE)
        optimizer.zero_grad()
        output=model(x)
        loss=nn.functional.cross_entropy(output,y)
        loss.backward()
        optimizer.step()
        pred=torch.argmax(output,dim=1).tolist()
        loss_list.append(loss)
        pred_list.extend(pred)
        fact_list.extend(y.tolist())
    accur=acurracy(pred_list,fact_list)
    print(f'epoch: {epo}')
    print(f'[train] accuracy: {accur}, loss: {sum(loss_list)/len(loss_list)}')
    with torch.no_grad():
        model.eval()
        fact_list=[]
        pred_list=[]
        loss_list=[]
        for x,y in test_dataloader:
            x,y=x.to(DEVICE),y.to(DEVICE)
            output=model(x)
            loss=nn.functional.cross_entropy(output,y).item()
            pred=torch.argmax(output,dim=1).tolist()
            loss_list.append(loss)
            pred_list.extend(pred)
            fact_list.extend(y.tolist())
        accur=acurracy(pred_list,fact_list)
        print(f'epoch: {epo}')
        print(f'[test] accuracy: {accur}, loss: {sum(loss_list)/len(loss_list)}')


AttributeError: 'list' object has no attribute 'ndim'